In [1]:
from nav2_simple_commander.robot_navigator import BasicNavigator

import rclpy as rp
from rclpy.node import Node
from rclpy.executors import MultiThreadedExecutor
from rclpy.duration import Duration

import tf2_ros
# from geometry_msgs.msg import PoseStamped
from geometry_msgs.msg import TransformStamped
from visualization_msgs.msg import MarkerArray

import time

import cv2
# import rclpy as rp
import numpy as np
import logging

# from rclpy.node import Node
from sensor_msgs.msg import Image
from geometry_msgs.msg import PoseStamped, PoseArray
from rcl_interfaces.msg import ParameterDescriptor, ParameterType
from visualization_msgs.msg import Marker, MarkerArray
from cv_bridge import CvBridge
from ament_index_python.packages import get_package_share_directory
from tf_transformations import quaternion_from_matrix

In [2]:
def move_with_long_delay(arg_x, arg_y, arg_w, arg_z):
    
    print(f"Move to pose: x {arg_x}, y {arg_y}, orientation: w {arg_w}, z {arg_z}")
    
    goal_pose1 = PoseStamped()
    goal_pose1.header.frame_id = 'map'
    goal_pose1.header.stamp = nav.get_clock().now().to_msg()
    goal_pose1.pose.position.x = arg_x
    goal_pose1.pose.position.y = arg_y
    goal_pose1.pose.orientation.w = arg_w
    goal_pose1.pose.orientation.z = arg_z

    nav.goToPose(goal_pose1)

    i = 0
    while not nav.isTaskComplete():
        i = i + 1

        feedback = nav.getFeedback()
        if feedback and i % 5 == 0:
            print('Distance remaining: ' + '{:.2f}'.format(
            feedback.distance_remaining) + 'meters.')
            print(feedback.navigation_time)

        if Duration.from_msg(feedback.navigation_time) > Duration(seconds=5.0):
            nav.cancelTask()
            

def move_with_short_delay(arg_x, arg_y, arg_w, arg_z):
    
    print(f"Move to pose: x {arg_x}, y {arg_y}, orientation: w {arg_w}, z {arg_z}")
    
    goal_pose1 = PoseStamped()
    goal_pose1.header.frame_id = 'map'
    goal_pose1.header.stamp = nav.get_clock().now().to_msg()
    goal_pose1.pose.position.x = arg_x
    goal_pose1.pose.position.y = arg_y
    goal_pose1.pose.orientation.w = arg_w
    goal_pose1.pose.orientation.z = arg_z

    nav.goToPose(goal_pose1)

    i = 0
    while not nav.isTaskComplete():
        i = i + 1

        feedback = nav.getFeedback()
        if feedback and i % 5 == 0:
            print('Distance remaining: ' + '{:.2f}'.format(
            feedback.distance_remaining) + 'meters.')
            print(feedback.navigation_time)

        if Duration.from_msg(feedback.navigation_time) > Duration(seconds=3.0):
            nav.cancelTask()

In [3]:
# class MarkerChecker(Node):
#     def __init__(self):
#         super().__init__('marker_checker')

#         self.marker_sub = self.create_subscription(
#             MarkerArray,
#             '/marker_pose',
#             self.marker_callback,
#             10
#         )
#         self.is_detected = False
#         self.marker_id = -1

#     def marker_callback(self, msg):
#         self.is_detected = False
#         self.marker_id = -1
#         for marker in msg.markers:
#             print(f"marker.id: {marker.id}")
#             self.is_detected = True
#             self.marker_id = marker.id
            

In [4]:
class MarkerDetector(Node):

    def __init__(self):
        super().__init__('marker_5x5_node')
        self.distance = 0.0
        self.x = 0.0
        self.y = 0.0
        self.cv_bridge = CvBridge()

        self.calibration_path = get_package_share_directory('aruco_pkg') + '/params/camera_calibration.npz'
        self.calib_data = np.load(self.calibration_path)
        self.cam_mat = self.calib_data["camera_matrix"]
        self.dist_coef = self.calib_data["distortion_coefficients"]
        self.r_vectors = self.calib_data["rvecs"]
        self.t_vectors = self.calib_data["tvecs"]

        self.declare_parameter('marker_size', 3.5,
descriptor=ParameterDescriptor(type=ParameterType.PARAMETER_DOUBLE))
        self.marker_size_str = self.get_parameter('marker_size').value
        self.marker_size = float(self.marker_size_str)
        self.get_logger().info(f"Marker Size : {self.marker_size}")
        
        self.aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_5X5_250)
        self.aruco_params = cv2.aruco.DetectorParameters_create()

        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger("marker_5x5_node")

        self.camera_sub = self.create_subscription(
            Image,
            'image_raw',
            self.image_callback,
            10
        )
        
        self.is_detected = False
        self.marker_id = -1
        
        self.pose_publisher = self.create_publisher(MarkerArray, 'marker_pose', 10)
    #convert from 3x3 matrix to quaternion
    
    def image_callback(self, msg):
        if msg is None:
            self.get_logger().warn("No Camera Data has been received!")
        cv_image = self.cv_bridge.imgmsg_to_cv2(msg, desired_encoding='rgb8')
        
        gray = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)
        corners, ids, _ = cv2.aruco.detectMarkers(gray, self.aruco_dict,
parameters=self.aruco_params)

        pose_array = PoseArray()

        marker_array = MarkerArray()

        if corners:
            rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(
                corners, self.marker_size, self.cam_mat, self.dist_coef
            )

            total_markers = range(0, ids.size)
            for id, corner, i in zip(ids, corners, total_markers):
                


                rotation_matrix, _ = cv2.Rodrigues(rvecs[0])

                self.get_logger().info(f"R : {rotation_matrix}")
                
                transform_matrix = np.eye(4)
                transform_matrix[:3, :3] = rotation_matrix
                
                quaternion = quaternion_from_matrix(transform_matrix)
                pose_msg = PoseStamped()
                pose_msg.header.stamp = self.get_clock().now().to_msg()
                pose_msg.header.frame_id = 'base_link'
                pose_msg.pose.position.x = tvecs[0][0][0]
                pose_msg.pose.position.y = tvecs[0][0][1]
                pose_msg.pose.position.z = tvecs[0][0][2]

                pose_msg.pose.orientation.x = quaternion[0]
                pose_msg.pose.orientation.y = quaternion[1]
                pose_msg.pose.orientation.z = quaternion[2]
                pose_msg.pose.orientation.w = quaternion[3]

                pose_array.header = pose_msg.header

                pose_array.poses.append(pose_msg.pose)
                # self.pose_publisher.publish(pose_array)

                marker_msg = Marker()
                marker_msg.header.frame_id = pose_msg.header.frame_id
                marker_msg.header.stamp = pose_msg.header.stamp
                marker_msg.id = int(id[0])
                marker_msg.type = Marker.TEXT_VIEW_FACING
                marker_msg.pose = pose_msg.pose
                marker_msg.scale.z = 0.05
                marker_msg.color.r = 1.0
                marker_msg.color.g = 1.0
                marker_msg.color.b = 1.0
                marker_msg.color.a = 1.0

                marker_array.markers.append(marker_msg)
                
                
                self.is_detected = True
                self.marker_id = marker_msg.id
                print("Aruco Marker is detected!")
                print(f"The maker_id: {self.marker_id}")
                break

In [5]:
rp.init()

nav = BasicNavigator()
nav.waitUntilNav2Active()

[INFO] [1691050554.076955280] [basic_navigator]: Nav2 is ready for use!


In [6]:
# executor = MultiThreadedExecutor()

# mc = MarkerChecker()
md = MarkerDetector()

# executor.add_node(mc)
# executor.add_node(md)

# try:
#     executor.spin()
    
while True:
    rp.spin_once(md)
    # go to A
    print("Move to A")
    move_with_long_delay(1.15, 0.15, 0., 0.) # to A

    if md.is_detected == True and md.marker_id == 10:
        print("Detected marker in A")
        # turn back
        print("Turning back")
        move_with_short_delay(1.15, 0.15, 0., 1.) # turn 180
        break

while True:
    rp.spin_once(md)
    # go to C
    print("Move to C")
    move_with_long_delay(1.5, 1., 1., 1.) # to C

    if md.is_detected == True and md.marker_id == 5:
        print("Detected marker in C")
        print("Turning spin")
        # robot spin
        move_with_short_delay(1.5, 1., 1., 1.)
        move_with_short_delay(1.5, 1., 0., 0.)
        move_with_short_delay(1.5, 1., 1., -1.)
        move_with_short_delay(1.5, 1., 0., 1.)
        move_with_short_delay(1.5, 1., 1., 1.)
        break

while True:
    rp.spin_once(md)
    print("Move to B")
    # go to B
    move_with_long_delay(1.7, -0.15, 0., 0.) # to B

    if md.is_detected == True and md.marker_id == 15:
        print("Detected marker in B")
        # print end
        print("End!!!!!!!")
        break


[INFO] [1691050559.622669938] [marker_5x5_node]: Marker Size : 3.5
[INFO] [1691050559.632301524] [basic_navigator]: Navigating to goal: 1.15 0.15...


Move to A
Move to pose: x 1.15, y 0.15, orientation: w 0.0, z 0.0
Distance remaining: 1.17meters.
builtin_interfaces.msg.Duration(sec=0, nanosec=500442401)
Distance remaining: 1.12meters.
builtin_interfaces.msg.Duration(sec=1, nanosec=486649)
Distance remaining: 1.07meters.
builtin_interfaces.msg.Duration(sec=1, nanosec=500461038)
Distance remaining: 0.92meters.
builtin_interfaces.msg.Duration(sec=2, nanosec=401547)
Distance remaining: 0.82meters.
builtin_interfaces.msg.Duration(sec=2, nanosec=510372233)
Distance remaining: 0.77meters.
builtin_interfaces.msg.Duration(sec=3, nanosec=10376081)
Distance remaining: 0.67meters.
builtin_interfaces.msg.Duration(sec=3, nanosec=510375946)
Distance remaining: 0.67meters.
builtin_interfaces.msg.Duration(sec=4, nanosec=10379393)
Distance remaining: 0.52meters.
builtin_interfaces.msg.Duration(sec=4, nanosec=520400653)
Distance remaining: 0.42meters.
builtin_interfaces.msg.Duration(sec=5, nanosec=30284779)
Move to A
Move to pose: x 1.15, y 0.15, ori

[INFO] [1691050564.667694864] [basic_navigator]: Canceling current task.
[INFO] [1691050564.698435318] [basic_navigator]: Navigating to goal: 1.15 0.15...


Distance remaining: 0.24meters.
builtin_interfaces.msg.Duration(sec=0, nanosec=490128184)
Distance remaining: 0.17meters.
builtin_interfaces.msg.Duration(sec=0, nanosec=990154065)
Distance remaining: 0.22meters.
builtin_interfaces.msg.Duration(sec=1, nanosec=500170378)
Distance remaining: 0.22meters.
builtin_interfaces.msg.Duration(sec=2, nanosec=115081)
Distance remaining: 0.22meters.
builtin_interfaces.msg.Duration(sec=2, nanosec=500151787)
Distance remaining: 0.22meters.
builtin_interfaces.msg.Duration(sec=3, nanosec=182555)
Distance remaining: 0.22meters.
builtin_interfaces.msg.Duration(sec=3, nanosec=510209235)
Distance remaining: 0.22meters.
builtin_interfaces.msg.Duration(sec=4, nanosec=10163810)
Distance remaining: 0.22meters.
builtin_interfaces.msg.Duration(sec=4, nanosec=510165804)
Distance remaining: 0.22meters.
builtin_interfaces.msg.Duration(sec=5, nanosec=10171460)
Move to A
Move to pose: x 1.15, y 0.15, orientation: w 0.0, z 0.0
Aruco Marker is detected!
The maker_id: 10

[INFO] [1691050569.732848794] [basic_navigator]: Canceling current task.
[INFO] [1691050569.748299300] [basic_navigator]: Navigating to goal: 1.15 0.15...
[INFO] [1691050569.800241673] [marker_5x5_node]: R : [[ 0.05588252  0.99775556 -0.03689162]
 [ 0.99832713 -0.05528919  0.0169128 ]
 [ 0.01483513 -0.03777504 -0.99917614]]


...

[INFO] [1691050617.536716984] [basic_navigator]: Canceling current task.
[INFO] [1691050617.547553307] [basic_navigator]: Navigating to goal: 1.7 -0.15...


Distance remaining: 0.10meters.
builtin_interfaces.msg.Duration(sec=0, nanosec=500180794)
Distance remaining: 0.10meters.
builtin_interfaces.msg.Duration(sec=1, nanosec=232539)
Distance remaining: 0.10meters.
builtin_interfaces.msg.Duration(sec=1, nanosec=510176008)
Distance remaining: 0.10meters.
builtin_interfaces.msg.Duration(sec=2, nanosec=10293232)
Distance remaining: 0.05meters.
builtin_interfaces.msg.Duration(sec=2, nanosec=510176351)
Distance remaining: 0.05meters.
builtin_interfaces.msg.Duration(sec=3, nanosec=20795674)
Distance remaining: 0.05meters.
builtin_interfaces.msg.Duration(sec=3, nanosec=520202486)
Distance remaining: 0.05meters.
builtin_interfaces.msg.Duration(sec=4, nanosec=20224530)
Distance remaining: 0.05meters.
builtin_interfaces.msg.Duration(sec=4, nanosec=520719883)
Distance remaining: 0.05meters.
builtin_interfaces.msg.Duration(sec=5, nanosec=30141821)
Move to B
Move to pose: x 1.7, y -0.15, orientation: w 0.0, z 0.0
Aruco Marker is detected!
The maker_id: 1

[INFO] [1691050622.579368489] [basic_navigator]: Canceling current task.
[INFO] [1691050622.592423229] [basic_navigator]: Navigating to goal: 1.7 -0.15...
[INFO] [1691050622.600265030] [marker_5x5_node]: R : [[-0.98986166  0.10270518  0.09810987]
 [ 0.09665113  0.99321992 -0.06459675]
 [-0.1040791  -0.05445942 -0.99307689]]


Distance remaining: 0.05meters.
builtin_interfaces.msg.Duration(sec=0, nanosec=510177457)
Distance remaining: 0.05meters.
builtin_interfaces.msg.Duration(sec=1, nanosec=10213606)
Distance remaining: 0.05meters.
builtin_interfaces.msg.Duration(sec=1, nanosec=510179092)
Distance remaining: 0.05meters.
builtin_interfaces.msg.Duration(sec=2, nanosec=20358891)
Distance remaining: 0.05meters.
builtin_interfaces.msg.Duration(sec=2, nanosec=520184242)
Distance remaining: 0.05meters.
builtin_interfaces.msg.Duration(sec=3, nanosec=20334788)
Distance remaining: 0.05meters.
builtin_interfaces.msg.Duration(sec=3, nanosec=530247648)
Distance remaining: 0.05meters.
builtin_interfaces.msg.Duration(sec=4, nanosec=30184584)
Distance remaining: 0.05meters.
builtin_interfaces.msg.Duration(sec=4, nanosec=540133879)
Distance remaining: 0.05meters.
builtin_interfaces.msg.Duration(sec=5, nanosec=40166907)
Detected marker in B
End!!!!!!!


[INFO] [1691050627.639342259] [basic_navigator]: Canceling current task.


In [ ]:
# move_with_long_delay(1.7, 0., 0., 0.)


# move_with_long_delay(1.0, 0.2, 0., 0.) # to A
# move_with_long_delay(1.7, -0.15, 0., 0.) # to B
move_with_long_delay(1.5, 1., 1., 1.) # to C

In [ ]:
rp.shutdown()

In [ ]:
# # initial setting
# goal_pose0 = PoseStamped()
# goal_pose0.header.frame_id = 'map'
# goal_pose0.header.stamp = nav.get_clock().now().to_msg()
# goal_pose0.pose.position.x = 100.
# goal_pose0.pose.position.y = 100.
# goal_pose0.pose.orientation.w = 0.
# goal_pose0.pose.orientation.z = 0.

# nav.goToPose(goal_pose0)

# i = 0
# while not nav.isTaskComplete():
#     i = i + 1
#     feedback = nav.getFeedback()
#     if Duration.from_msg(feedback.navigation_time) > Duration(seconds=5.0):
#         nav.cancelTask()

        
# A area
goal_pose1 = PoseStamped()
goal_pose1.header.frame_id = 'map'
goal_pose1.header.stamp = nav.get_clock().now().to_msg()
goal_pose1.pose.position.x = 0.7
goal_pose1.pose.position.y = 0.2
goal_pose1.pose.orientation.w = 0.
goal_pose1.pose.orientation.z = 0.

nav.goToPose(goal_pose1)

i = 0
while not nav.isTaskComplete():
    i = i + 1
    feedback = nav.getFeedback()
    if feedback and i % 5 == 0:
        print('Distance remaining: ' + '{:.2f}'.format(
        feedback.distance_remaining) + 'meters.')
        print(feedback.navigation_time)
        
    if Duration.from_msg(feedback.navigation_time) > Duration(seconds=7.0):
        nav.cancelTask()

        
# turning right in A area
goal_pose2 = PoseStamped()
goal_pose2.header.frame_id = 'map'
goal_pose2.header.stamp = nav.get_clock().now().to_msg()
goal_pose2.pose.position.x = 0.7
goal_pose2.pose.position.y = 0.2
goal_pose2.pose.orientation.w = 1.
goal_pose2.pose.orientation.z = -1.

nav.goToPose(goal_pose2)

i = 0
while not nav.isTaskComplete():
    i = i + 1
    feedback = nav.getFeedback()
    if feedback and i % 5 == 0:
        print('Distance remaining: ' + '{:.2f}'.format(
        feedback.distance_remaining) + 'meters.')
        print(feedback.navigation_time)
        
    if Duration.from_msg(feedback.navigation_time) > Duration(seconds=5.0):
        nav.cancelTask()


# C area
goal_pose3 = PoseStamped()
goal_pose3.header.frame_id = 'map'
goal_pose3.header.stamp = nav.get_clock().now().to_msg()
goal_pose3.pose.position.x = 1.7
goal_pose3.pose.position.y = 1.
goal_pose3.pose.orientation.w = 0.
goal_pose3.pose.orientation.z = 0.

nav.goToPose(goal_pose3)

i = 0
while not nav.isTaskComplete():
    i = i + 1
    feedback = nav.getFeedback()
    if feedback and i % 5 == 0:
        print('Distance remaining: ' + '{:.2f}'.format(
        feedback.distance_remaining) + 'meters.')
        print(feedback.navigation_time)
        
    if Duration.from_msg(feedback.navigation_time) > Duration(seconds=10.0):
        nav.cancelTask()

# stopping 1 second
time.sleep(1)


# B area
goal_pose4 = PoseStamped()
goal_pose4.header.frame_id = 'map'
goal_pose4.header.stamp = nav.get_clock().now().to_msg()
goal_pose4.pose.position.x = 1.7
goal_pose4.pose.position.y = -0.15
goal_pose4.pose.orientation.w = 0.
goal_pose4.pose.orientation.z = 0.

nav.goToPose(goal_pose4)

i = 0
while not nav.isTaskComplete():
    i = i + 1
    feedback = nav.getFeedback()
    if feedback and i % 5 == 0:
        print('Distance remaining: ' + '{:.2f}'.format(
        feedback.distance_remaining) + 'meters.')
        print(feedback.navigation_time)
        
    if Duration.from_msg(feedback.navigation_time) > Duration(seconds=10.0):
        nav.cancelTask()

In [ ]:
# initial point

goal_pose1 = PoseStamped()
goal_pose1.header.frame_id = 'map'
goal_pose1.header.stamp = nav.get_clock().now().to_msg()
goal_pose1.pose.position.x = 0.
goal_pose1.pose.position.y = 0.
goal_pose1.pose.orientation.w = 0.
goal_pose1.pose.orientation.z = 0.

nav.goToPose(goal_pose1)

i = 0
while not nav.isTaskComplete():
    i = i + 1
    feedback = nav.getFeedback()
    if feedback and i % 5 == 0:
        print('Distance remaining: ' + '{:.2f}'.format(
        feedback.distance_remaining) + 'meters.')
        print(feedback.navigation_time)
        
    if Duration.from_msg(feedback.navigation_time) > Duration(seconds=7.0):
        nav.cancelTask()